In [ ]:
import pandas as pd
import numpy as np
from collections import namedtuple
from model.deepmove import DeepmoveTrainer, DeepmoveForecaster
from warnings import filterwarnings
import matplotlib.pyplot as plt

filterwarnings('ignore')
import torch
Parameters = namedtuple('Parameters', [
    'loc_size', 'loc_emb_size', 'tim_size', 'tim_emb_size', 'user_size', 'user_emb_size',
    'hidden_size', 'attn_type', 'use_cuda', 'rnn_type', 'dropout_p'
])
df = pd.read_csv('./data/cityD-dataset.csv')

# Split day
SPLIT_D = 60
# Number of users
USER_NUM = 3
# Predicting days
FUTURE_DAYS = 15
EPOCHS = 40

# Embedding sizes
HIDDEN_SIZE = 16
TIM_EMB_SIZE = 128
LOC_EMB_SIZE = 400
USER_EMB_SIZE = 128

target_df = df[df.uid.isin(df.uid.drop_duplicates()[:USER_NUM])]
training_set = target_df[target_df.d < SPLIT_D]
testing_set = target_df[target_df.d >= SPLIT_D]

parameters = Parameters(loc_size=40100, loc_emb_size=LOC_EMB_SIZE, tim_size=48, tim_emb_size=TIM_EMB_SIZE,
                        user_size=USER_NUM, user_emb_size=USER_EMB_SIZE, hidden_size=HIDDEN_SIZE, attn_type='general',
                        use_cuda=torch.cuda.is_available(), rnn_type='GRU', dropout_p=0.1)
trainer = DeepmoveTrainer(parameters)
trainer.train(training_set, epochs=EPOCHS, batch_size=64)

2025-10-16 01:12:25 - INFO - Creating TrajectoryDataset...
Processing users: 100%|██████████| 3/3 [00:03<00:00,  1.16s/it]
2025-10-16 01:12:29 - INFO - Created 175 training samples from 3 users
Epoch 1/40: 100%|██████████| 3/3 [00:04<00:00,  1.60s/it, loss=10.4859]


Epoch 1/40, Avg Loss: 10.5535, Time: 4.80s


Epoch 2/40: 100%|██████████| 3/3 [00:03<00:00,  1.16s/it, loss=10.2742]


Epoch 2/40, Avg Loss: 10.3603, Time: 3.50s


Epoch 3/40: 100%|██████████| 3/3 [00:03<00:00,  1.18s/it, loss=10.1173]


...

Epoch 6/40: 100%|██████████| 3/3 [00:03<00:00,  1.12s/it, loss=9.6174]


Epoch 6/40, Avg Loss: 9.7411, Time: 3.36s


Epoch 7/40: 100%|██████████| 3/3 [00:03<00:00,  1.24s/it, loss=9.4926]


Epoch 7/40, Avg Loss: 9.5853, Time: 3.74s


Epoch 8/40:  33%|███▎      | 1/3 [00:01<00:02,  1.31s/it, loss=9.5508]

In [ ]:

forecaster = DeepmoveForecaster(trainer.model, parameters)
predictions = forecaster.forecast(training_set, predicting_days=FUTURE_DAYS)

In [ ]:

uid = 2
pred = testing_set[['uid', 'd', 't']].merge(predictions, on=['uid', 'd', 't'], how='left').dropna()
target_pred = pred[pred['uid'] == uid].reset_index(drop=True)
target_test = testing_set[(testing_set['uid'] == uid) & (testing_set.d.isin(target_pred.d))].reset_index(drop=True)

# Function to create the time label in the format d:HH:MM
def get_time_label(row):
    t = row['t']
    hh = t // 2
    mm = 30 if t % 2 == 1 else 0
    return f"{int(row['d'])}:{hh:02d}:{mm:02d}"

# Apply the function to create time labels (assuming same time points for pred and test)
target_pred['time_label'] = target_pred.apply(get_time_label, axis=1)
target_test['time_label'] = target_test.apply(get_time_label, axis=1)

# Create separate plots for x and y vs. time index
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 8), sharex=True)

# Plot for x (predicted and testing/actual)
ax1.plot(range(len(target_pred)), target_pred['x'], marker='o', linestyle='-', color='blue', label='predicted x')
ax1.plot(range(len(target_test)), target_test['x'], marker='x', linestyle='--', color='green', label='actual x')
ax1.set_ylabel('x')
ax1.set_title('Predicted vs Actual x over Time')
ax1.grid(True)
ax1.legend()

# Plot for y (predicted and testing/actual)
ax2.plot(range(len(target_pred)), target_pred['y'], marker='o', linestyle='-', color='red', label='predicted y')
ax2.plot(range(len(target_test)), target_test['y'], marker='x', linestyle='--', color='orange', label='actual y')
ax2.set_ylabel('y')
ax2.set_title('Predicted vs Actual y over Time')
ax2.grid(True)
ax2.legend()

# Set x-axis ticks and labels using pred's time labels (assuming alignment)
plt.xticks(range(len(target_pred)), target_pred['time_label'], rotation=45, ha='right')
plt.xlabel('Time (d:HH:MM)')

plt.tight_layout()
plt.show()